# RedPanda Examples

In [1]:
import pandas, redpanda, sqlalchemy, sys
print "RedPanda version:", redpanda.__version__
print "Python version:", sys.version

RedPanda version: 0.2.4
Python version: 2.7.12 (default, Jun 29 2016, 08:57:23) 
[GCC 5.3.0]


## Opening a session

Open a `RedPanda` session to begin exploring data:

In [2]:
engine = redpanda.create_engine("sqlite://")
session = redpanda.Session(bind=engine)

Note that this is syntactic sugar for SQLAlchemy calls:

In [3]:
engine = sqlalchemy.create_engine("sqlite://")
sessionmaker = sqlalchemy.orm.sessionmaker(class_=redpanda.orm.Session,
                                           query_cls=redpanda.orm.Query)
session = sessionmaker(bind=engine)

## Creating An Example Model

Define your models using the declarative API. `RedPanda` defines one such model in the `example` module:

In [4]:
import sqlalchemy.ext.declarative
Base = sqlalchemy.ext.declarative.declarative_base()
class Widget(Base):
    id        = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    timestamp = sqlalchemy.Column(sqlalchemy.DateTime)
    name      = sqlalchemy.Column(sqlalchemy.String)
    kind      = sqlalchemy.Column(sqlalchemy.String)
    units     = sqlalchemy.Column(sqlalchemy.Integer)

    __read_sql__  = {"index_col": ["timestamp"], "parse_dates": ["timestamp"]}
    __tablename__ = "widgets"

Defining the class attribute `__read_sql__` will configure default arguments passed to `pandas.read_sql()`. 

In this case we are configuring the `widgets` table to use the `timestamp` column as an index and that we wish for `pandas` to parse this column as a `datetime` object.

In [5]:
from redpanda.example import create_widgets
create_widgets(session)

## Querying Data

Use the `frame()` method of a query to return the results as a `DataFrame`:

In [6]:
session.query(Widget).frame().head()

,id,name,kind,units
timestamp,,,,
2016-01-02 07:14:00,1,mats,buzzer,37
2016-01-03 11:21:00,2,cord,buzzer,91
2016-01-05 20:54:00,3,retirement,buzzer,5
2016-01-25 12:30:00,4,conflict,bopper,99
2016-01-27 11:03:00,5,laboratories,bopper,32


### Querying with Filters

Use SQLAlchemy filters to refine the query:

In [7]:
session.query(Widget).filter(Widget.timestamp.between("2016-11-01", "2016-11-30")).frame()

,id,name,kind,units
timestamp,,,,
2016-11-07 23:15:00,59,wagon,buzzer,17
2016-11-10 09:11:00,60,shovel,fizzer,46
2016-11-10 19:03:00,61,takeoffs,bopper,79
2016-11-18 14:20:00,62,matters,fizzer,15
2016-11-23 08:54:00,63,results,bopper,24
2016-11-23 23:43:00,64,reaches,bopper,14
2016-11-28 15:50:00,65,glass,buzzer,5


### Overriding Defaults

Pass arguments to `pandas.read_sql()` into the `frame()` method:

In [8]:
session.query(Widget).frame(index_col="id").head()

,timestamp,name,kind,units
id,,,,
1,2016-01-02 07:14:00,mats,buzzer,37
2,2016-01-03 11:21:00,cord,buzzer,91
3,2016-01-05 20:54:00,retirement,buzzer,5
4,2016-01-25 12:30:00,conflict,bopper,99
5,2016-01-27 11:03:00,laboratories,bopper,32


## Writing DataFrame to Session

Use the `add_dataframe()` method of the `RedPanda` session to place `DataFrame` rows into the session as models:

In [9]:
from datetime import datetime
dfdata = {
    "kind": "newkind", 
    "name": "newname", 
    "units": 1000, 
    "timestamp": datetime.utcnow()}
addme = pandas.DataFrame(dfdata, index=[0])
addme

,kind,name,timestamp,units
0,newkind,newname,2016-07-15 10:12:46.135608,1000


In [10]:
session.add_dataframe(Widget, addme)
session.commit()
session.query(Widget).filter(Widget.kind=="newkind").frame()

,id,name,kind,units
timestamp,,,,
2016-07-15 10:12:46.135608,76,newname,newkind,1000
